In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# default_exp runners

# Process

In [3]:
# export
from __future__ import annotations
from dataclasses import dataclass
from typing import Callable



In [4]:
# export
import logging
from tqdm.auto import tqdm

@dataclass
class Process:
    queries: list[Callable]
    steps: list[Callable]
    filter: Callable

# Simple in Memory Runner

In [5]:
# export

class RunnerMemory():
    def __init__(self, process: Process, progress_bar: bool = True):
        self.process = process
        self.progress_bar = progress_bar
        
    def query(self):
        for query in tqdm(self.process.queries, desc='query', disable=not self.progress_bar):
            for record in query.query():
                yield record

    def prepare(self, records):
        return self.process.filter(tqdm(records, desc='filter', disable=not self.progress_bar))

    def fetch(self, records):
        for record in tqdm(records, desc='fetch', disable=not self.progress_bar):
            yield (record.content, record)

    def transform(self, content_records):
        for content, record in tqdm(content_records, desc='transform', disable=not self.progress_bar):
            error = False
            for step in self.process.steps:
                try:
                    content = step(content, record)
                except Exception as e:
                    error = True
                    logging.error('Error processing %s at step %s: %s' % (record, step.__name__, e))
                    break
            if not error:
                yield content

    def run(self):
        records = self.prepare(self.query())
        content_records = self.fetch(records)
        return self.transform(content_records) 

Minibatch : too small leads to time overhead on setting up connections, too large can lead to memory issues.

In [62]:
# export
import itertools
from pathlib import Path
from sqlitedict import SqliteDict

def minibatch(seq, size):
    items = []
    for x in seq:
        items.append(x)
        if len(items) >= size:
            yield items
            items = []
    if items:
        yield items
        
import zlib, pickle, sqlite3
def compress_encode(obj: bytes):
     return sqlite3.Binary(zlib.compress(obj))
def compress_decode(obj):
     return zlib.decompress(bytes(obj))

class RunnerCached():
    def __init__(self, process: Process, path: Union[str, Path], progress_bar: bool = True, batch_size: int = 1024):
        self.process = process
        self.progress_bar = progress_bar
        self.batch_size = batch_size
        
        self.path = Path(path)
        
        self._query = SqliteDict(path, tablename='query', autocommit=True)
        self._fetch = SqliteDict(path, tablename='fetch', autocommit=False, encode=compress_encode, decode=compress_decode)
        
    def query(self):
        # TODO: Don't cache WaybackQuery or FileQuery
        for query in tqdm(self.process.queries, desc='query', disable=not self.progress_bar):
            key = repr(query)
            if key not in self._query:
                self._query[key] = list(query.query())
                
        # TODO: Merge
        for query in self.process.queries:
            for record in self._query[key]:
                yield record
                
    def prepare(self, records):
        return self.process.filter(tqdm(records, desc='filter', disable=not self.progress_bar))

    def fetch(self, records):
        records = list(records)
        unfetched_records = sorted([r for r in records if r.digest not in self._fetch], key=lambda x: str(type(x)))
        
        with tqdm(total=len(unfetched_records), desc='fetch') as pbar:
            for cls, record_group in itertools.groupby(unfetched_records, key=type):
                for record_group_batch in minibatch(record_group, self.batch_size):
                    record_group_batch = list(record_group_batch)
                    for content, record in zip(cls.fetch_parallel(record_group_batch, 
                                                                  callback=lambda r, c: pbar.update(1)),
                                               record_group_batch):
                        assert record.digest is not None
                        self._fetch[record.digest] = content
                    self._fetch.commit()

        for record in records:
            yield (self._fetch[record.digest], record)
                

    def transform(self, content_records):
        for content, record in tqdm(content_records, desc='transform', disable=not self.progress_bar):
            error = False
            for step in self.process.steps:
                try:
                    content = step(content, record)
                except Exception as e:
                    error = True
                    logging.error('Error processing %s at step %s: %s' % (record, step.__name__, e))
                    break
            if not error:
                yield content

    def run(self):
        records = self.prepare(self.query())
        content_records = self.fetch(records)
        return self.transform(content_records) 

## Simple test process

In practice we'd use a something like parsel, beautifulsoup or selectolax.
However for a simple demo this has no external dependencies.

In [63]:
from html.parser import HTMLParser

class SkeptricHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.extract = {}
        self.field = None
        
    def handle_starttag(self, tag, attrs):
        if dict(attrs).get('class') == 'post-full-title':
            self.field = 'title'
        if dict(attrs).get('class') == 'byline-meta-date':
            self.field = 'date'

    def handle_endtag(self, tag):
        self.field = None

    def handle_data(self, data):
        if self.field is not None:
            self.extract[self.field] = data

def skeptric_filter(records):
    for r in records:
        if r.mime == 'text/html' and r.status == 200:
            yield r
        
def skeptric_extract(content, metadata):
    parser = SkeptricHTMLParser()
    html = content.decode('utf-8')
    parser.feed(html)
    data = parser.extract
    data['url'] = metadata.url
    data['timestamp'] = metadata.timestamp
    return data

def skeptric_verify_extract(content, metadata):
    if not content.get('title'):
        raise ValueError('Missing title')
    if not content.get('date'):
        raise ValueError('Missing date')
    return content

from datetime import datetime
def skeptric_normalise(content, metadata):
    content = content.copy()
    content['date'] = datetime.strptime(content['date'], '%d %B %Y')
    return content

from webrefine.query import WarcFileQuery
test_data = '../resources/test/skeptric.warc.gz'

skeptric_query = WarcFileQuery(test_data)

# Test Memory Runner

In [64]:
skeptric_process = Process(queries=[skeptric_query],
                     filter=skeptric_filter,
                     steps=[skeptric_extract, skeptric_verify_extract, skeptric_normalise])

In [65]:
data = list(RunnerMemory(skeptric_process).run())
data

filter: 0it [00:00, ?it/s]

transform: 0it [00:00, ?it/s]

fetch: 0it [00:00, ?it/s]

query:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:root:Error processing WarcFileRecord(url='https://skeptric.com/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 36), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=17122, digest='JJVB3MQERHRZJCHOJNKS5VDOODXPZAV2') at step skeptric_verify_extract: Missing title
ERROR:root:Error processing WarcFileRecord(url='https://skeptric.com/tags/data/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 38), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=130269, digest='R7CLAACFU5L7T5LKI5G53RZSMCNUNV6F') at step skeptric_verify_extract: Missing title


[{'title': "Pagination in Internet Archive's Wayback Machine with CDX",
  'date': datetime.datetime(2021, 11, 23, 0, 0),
  'url': 'https://skeptric.com/pagination-wayback-cdx/',
  'timestamp': datetime.datetime(2021, 11, 26, 11, 28, 34)},
 {'title': 'About Skeptric',
  'date': datetime.datetime(2021, 10, 18, 0, 0),
  'url': 'https://skeptric.com/about/',
  'timestamp': datetime.datetime(2021, 11, 26, 11, 28, 37)},
 {'title': 'Searching 100 Billion Webpages Pages With Capture Index',
  'date': datetime.datetime(2020, 6, 11, 0, 0),
  'url': 'https://skeptric.com/searching-100b-pages-cdx/',
  'timestamp': datetime.datetime(2021, 11, 26, 11, 28, 39)},
 {'title': 'Fast Web Dataset Extraction Worfklow',
  'date': datetime.datetime(2021, 11, 21, 0, 0),
  'url': 'https://skeptric.com/fast-web-data-workflow/',
  'timestamp': datetime.datetime(2021, 11, 26, 11, 28, 39)},
 {'title': 'Unique Key for Web Captures',
  'date': datetime.datetime(2021, 11, 19, 0, 0),
  'url': 'https://skeptric.com/key-

We can always look up an error

Would be nicer if everything was a string so we didn't have to handle the imports...

In [66]:
from webrefine.query import WarcFileRecord
from pathlib import PosixPath
WarcFileRecord(url='https://skeptric.com/', timestamp=datetime(2021, 11, 26, 11, 28, 36), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=17122, digest='JJVB3MQERHRZJCHOJNKS5VDOODXPZAV2')

WarcFileRecord(url='https://skeptric.com/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 36), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=17122, digest='JJVB3MQERHRZJCHOJNKS5VDOODXPZAV2')

# Test Persistent Runner

In [67]:
%%time
from pathlib import Path
test_cache_path = Path('./test_skeptric_cache.sqlite')

if test_cache_path.exists():
    test_cache_path.unlink()

data_cached = list(RunnerCached(skeptric_process, test_cache_path).run())

filter: 0it [00:00, ?it/s]

transform: 0it [00:00, ?it/s]

query:   0%|          | 0/1 [00:00<?, ?it/s]

fetch:   0%|          | 0/8 [00:00<?, ?it/s]

ERROR:root:Error processing WarcFileRecord(url='https://skeptric.com/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 36), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=17122, digest='JJVB3MQERHRZJCHOJNKS5VDOODXPZAV2') at step skeptric_verify_extract: Missing title
ERROR:root:Error processing WarcFileRecord(url='https://skeptric.com/tags/data/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 38), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=130269, digest='R7CLAACFU5L7T5LKI5G53RZSMCNUNV6F') at step skeptric_verify_extract: Missing title


CPU times: user 794 ms, sys: 12.6 ms, total: 807 ms
Wall time: 799 ms


In [68]:
assert data_cached == data

In [77]:
assert data_cached == list(RunnerCached(skeptric_process, test_cache_path).run())

filter: 0it [00:00, ?it/s]

transform: 0it [00:00, ?it/s]

query:   0%|          | 0/1 [00:00<?, ?it/s]

fetch:   0%|          | 0/8 [00:00<?, ?it/s]

ERROR:root:Error processing WarcFileRecord(url='https://skeptric.com/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 36), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=17122, digest='JJVB3MQERHRZJCHOJNKS5VDOODXPZAV2') at step skeptric_verify_extract: Missing title
ERROR:root:Error processing WarcFileRecord(url='https://skeptric.com/tags/data/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 38), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=130269, digest='R7CLAACFU5L7T5LKI5G53RZSMCNUNV6F') at step skeptric_verify_extract: Missing title


In [69]:
%%time
from pathlib import Path
test_cache_path = Path('./test_skeptric_cache.sqlite')

if test_cache_path.exists():
    test_cache_path.unlink()

data_cached_small_batch = list(RunnerCached(skeptric_process, test_cache_path, batch_size=2).run())

filter: 0it [00:00, ?it/s]

transform: 0it [00:00, ?it/s]

query:   0%|          | 0/1 [00:00<?, ?it/s]

fetch:   0%|          | 0/8 [00:00<?, ?it/s]

ERROR:root:Error processing WarcFileRecord(url='https://skeptric.com/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 36), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=17122, digest='JJVB3MQERHRZJCHOJNKS5VDOODXPZAV2') at step skeptric_verify_extract: Missing title
ERROR:root:Error processing WarcFileRecord(url='https://skeptric.com/tags/data/', timestamp=datetime.datetime(2021, 11, 26, 11, 28, 38), mime='text/html', status=200, path=PosixPath('../resources/test/skeptric.warc.gz'), offset=130269, digest='R7CLAACFU5L7T5LKI5G53RZSMCNUNV6F') at step skeptric_verify_extract: Missing title


CPU times: user 777 ms, sys: 27.7 ms, total: 805 ms
Wall time: 788 ms


In [70]:
assert data_cached == data_cached_small_batch

In [72]:
#slow
from webrefine.query import CommonCrawlQuery, WaybackQuery
from datetime import datetime

skeptric_cc = CommonCrawlQuery('skeptric.com/*', apis=['CC-MAIN-2021-43'])

skeptric_wb = WaybackQuery('skeptric.com/*', start='202103', end='202111')

def skeptric_filter_strict(records):
    for r in records:
        if r.mime != 'text/html' or r.status != 200:
            continue
        if '/tags/' in r.url or '/notebooks/' in r.url or r.url.endswith('skeptric.com/'):
            continue
        yield r

skeptric_process_all = Process(queries=[skeptric_query, skeptric_cc, skeptric_wb],
                     filter=skeptric_filter_strict,
                     steps=[skeptric_extract, skeptric_verify_extract, skeptric_normalise])

The cached runner has to evaluate everything the first time and so is slow.

TODO: We need to fix the fetch progress bar (callbacks?)

In [73]:
# slow
%time data_all = list(RunnerCached(skeptric_process_all, test_cache_path).run())
len(data_all)

filter: 0it [00:00, ?it/s]

transform: 0it [00:00, ?it/s]

query:   0%|          | 0/3 [00:00<?, ?it/s]

fetch:   0%|          | 0/873 [00:00<?, ?it/s]

CPU times: user 20.3 s, sys: 1.41 s, total: 21.7 s
Wall time: 1min 13s


882

Cache size in MB

In [74]:
test_cache_path.stat().st_size / 1024**2

1.828125

It runs much faster the second time

In [75]:
# slow
%time data_all_2 = list(RunnerCached(skeptric_process_all, test_cache_path).run())
assert data_all == data_all_2

filter: 0it [00:00, ?it/s]

transform: 0it [00:00, ?it/s]

query:   0%|          | 0/3 [00:00<?, ?it/s]

fetch: 0it [00:00, ?it/s]

CPU times: user 9.69 s, sys: 294 ms, total: 9.99 s
Wall time: 9.86 s


In [76]:
test_cache_path.unlink()